In [1]:
import os
import re
import subprocess
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
from IPython.display import clear_output

import dask.dataframe as dd
from tqdm.notebook import tqdm

read_csv_dtype = {'fasta': "object",
 'rank': "object",
 'ko_id': "object",
 'kegg_hit': "object",
 'peptidase_id': "object",
 'peptidase_family': "object",
 'peptidase_hit': "object",
 'peptidase_RBH': "object",
 'peptidase_identity': "float64",
 'peptidase_bitScore': "float64",
 'peptidase_eVal': "float64",
 'pfam_hits': "object",
 'cazy_ids': "object",
 'cazy_hits': "object",
 'cazy_subfam_ec': "object",
 'cazy_best_hit': "object",
 'vogdb_id': "object",
 'vogdb_hits': "object",
 'vogdb_categories': "object",
 'heme_regulatory_motif_count': "float64"}

Traditionally, methanogenesis and archaeal methanotrophy restricted to eight orders within Euryarchaeota:
- Methanopyrales
- Methanobacteriales
- Methanosarcinales
- Methanomassiliicoccales
- Candidatus Methanophagales (ANME-1)

![](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41579-018-0136-7/MediaObjects/41579_2018_136_Fig2_HTML.png?as=webp)

This is the Archaeal genome tree.

The blue ones indicates hydrogenotrophic/aceticlastic/methylotrophic Euryarchaeota:
- Methanopyrus kandleri
- Methanobacteriales
- Methanococcales
- Methanomicrobiales
- Ca. Methanoflorens stordalenmirensis
- Methanocellales
- Methanosarcinales
    - Methermicoccaceae
    - Methanosaetaceae
    - Methanosarcinaceae
    - Ca. Methanoperedens
- Ca. Methanophagales
- Ca. Syntrophoarchaeum

Yellow is methanotrophic Euryarchaeota:
- Ca. Methanoperedens
- Ca. Methanophagales

Red is alkanotrophs:
- Ca. Bathyarchaeota (TACK)
- Ca. Syntrophoarchaeum (Euryarchaeota)

Green is H2-dependent methylotrophic:
- Ca. Vestrearchaeota (TACK)
- Ca. Methanofastidiosa
- Methanonatronarchaeia
- Methanomassiliicoccales

Who has mcrA [KEGG: K00399] in my genomes?

In [2]:
summary_all = pd.read_csv("../../input_folder/summary_all.tsv", sep="\t", index_col=0)
summary_hq = pd.read_csv("../../input_folder/summary_hq.tsv", sep="\t", index_col=0)

genomes = summary_all.index.tolist()

In [3]:
summary_hq.head()

,Completeness,Contamination,Contig_N50,GC_Content,genome_size,average_coverage,number_of_contigs,16S_rRNA,23S_rRNA,5S_rRNA,classification,red_value
C1C3_F_metabat.43,96.65,2.03,98027,0.50,2863110,13.75,67,True,False,False,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-...,0.93561
C2E1_F_metabat.953,93.49,0.08,451638,0.52,2142996,15.33,7,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-...,0.93583
C1E5_M_semibin.11763,92.27,0.39,1841381,0.44,2607823,36.52,4,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-...,0.74397
C2E1_M_semibin.1716,90.52,2.37,1432256,0.45,2862093,156.84,4,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-...,0.74395
C3C4_F_semibin.3639,97.36,1.08,2729341,0.44,3054849,18.27,17,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-...,0.74317


In [4]:
def does_my_genome_have_this_kegg_gene_with_grep2(genome, kegg_gene):
    # Construct the file path
    file_path = f"../../input_folder/dram_faa/{genome}/annotations.tsv"
    
    # Use subprocess to run grep and capture output, looking for exact match in the 4th column
    result = subprocess.run(
        ["grep", "-P", "^([^\t]*\t){3}" + kegg_gene, file_path],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    
    # If grep found the gene, return True, otherwise False
    return result.returncode == 0  # grep returns 0 if found, non-zero if not

# Example usage
result = does_my_genome_have_this_kegg_gene_with_grep2(genomes[0], "K00399")
print(result)  # True if the KEGG gene exists in the 4th column, False otherwise


False


In [5]:
should_be_true = "C1E5_M_semibin.11763"
result = does_my_genome_have_this_kegg_gene_with_grep2(should_be_true, "K00399")
print(result)  # True if the KEGG gene exists in the 4th column, False otherwise

True


In [6]:
summary_all = pd.read_csv("../../input_folder/summary_all.tsv", sep="\t", index_col=0)
summary_hq = pd.read_csv("../../input_folder/summary_hq.tsv", sep="\t", index_col=0)
genomes = summary_all.index.tolist()
# genomes=["C1E5_M_semibin.11763"]

In [7]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def filter_genomes_for_kegg_gene(genomes, gene, max_workers=6):
    """Filter genomes to check if they contain a specific KEGG gene using multiprocessing."""
    filtered_genomes = []
    
    # Use ProcessPoolExecutor for multiprocessing
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the pool
        futures = {
            executor.submit(does_my_genome_have_this_kegg_gene_with_grep2, g, gene): g
            for g in genomes
        }

        # Use tqdm to track progress
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Filtering genomes for {gene}"):
            genome = futures[future]
            try:
                # Check the result of each task
                if future.result():
                    filtered_genomes.append(genome)
            except Exception as e:
                print(f"Error processing genome {genome}: {e}")

    return filtered_genomes

# Example usage
mcrA_genomes = filter_genomes_for_kegg_gene(genomes, "K00399", max_workers=6)

# mcrA_genomes = [
#     g for g in tqdm(genomes, desc="Filtering genomes for K00399")
#     if does_my_genome_have_this_kegg_gene_with_grep2(g, "K00399")
# ]

Filtering genomes for K00399:   0%|          | 0/2634 [00:00<?, ?it/s]

In [8]:
mcrA_genomes_hq = [g for g in mcrA_genomes if g in summary_hq.index.tolist()]
summary_hq.loc[mcrA_genomes_hq, :].style

,Completeness,Contamination,Contig_N50,GC_Content,genome_size,average_coverage,number_of_contigs,16S_rRNA,23S_rRNA,5S_rRNA,classification,red_value
C2E1_F_metabat.953,93.490000,0.080000,451638,0.520000,2142996,15.330000,7,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-38;f__Bog-38;g__Bog-38;s__,0.935830
C1E5_M_semibin.11763,92.270000,0.390000,1841381,0.440000,2607823,36.520000,4,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-38;f__Bog-38;g__;s__,0.743970
C2E1_M_semibin.1716,90.520000,2.370000,1432256,0.450000,2862093,156.840000,4,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-38;f__Bog-38;g__;s__,0.743950
C7F4_F_metabat.1057,94.390000,1.360000,1606116,0.430000,2651675,26.030000,2,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-38;f__Bog-38;g__;s__,0.744810
C7F4_M_semibin.4823,96.250000,0.480000,729249,0.440000,2922066,38.100000,8,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-38;f__Bog-38;g__;s__,0.740590
C3C4_F_semibin.3639,97.360000,1.080000,2729341,0.440000,3054849,18.270000,17,True,True,True,d__Archaea;p__Halobacteriota;c__Bog-38;o__Bog-38;f__Bog-38;g__;s__,0.743170
C3D5_M_semibin.1409,94.300000,0.670000,221283,0.620000,2141553,6.660000,26,True,True,True,d__Archaea;p__Halobacteriota;c__Methanomicrobia;o__Methanomicrobiales;f__JACTUA01;g__JACTUA01;s__,0.957050
C2E1_M_semibin.27400,94.470000,0.810000,2074696,0.600000,2074698,32.590000,1,True,True,True,d__Archaea;p__Halobacteriota;c__Methanomicrobia;o__Methanomicrobiales;f__JACTUA01;g__JACTUA01;s__,0.956900
C2E1_P_metabat.289,99.430000,2.060000,2242585,0.450000,2242705,13.780000,1,True,True,True,d__Archaea;p__Halobacteriota;c__Syntropharchaeia;o__Alkanophagales;f__Methanospirareceae;g__ANME-1-THS;s__,0.947890
C2D1_F_semibin.3899,100.000000,1.330000,1536192,0.290000,2983316,18.240000,19,True,True,True,d__Archaea;p__Methanobacteriota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobacterium_A;s__,0.916480


In [16]:
def remove_duplicates(input_list):
    seen = set()
    return [x for x in input_list if x not in seen and not seen.add(x)]

In [19]:
class KOHeatmapDashboard:
    def __init__(self, ko_list_path="../../input_folder/ko_list", ko_synonyms_path="../../input_folder/ko_synonyms"):
        # Load KO list and synonyms
        self.ko_list_df = self.load_ko_list(ko_list_path, ko_synonyms_path)
        self.ko_list = (self.ko_list_df.index + ": " + self.ko_list_df["gene_name"]).to_dict()
        self.ko_formatted_names = [
            i for i in self.ko_list_df.index + ": " + self.ko_list_df["gene_name"]
        ]

        # Custom color scheme
        self.colors = ["#e8e8e8", "#df0024", "#0085c7"]

        # Global matrix for persistence
        self.df_count_ko = pd.DataFrame()

        # Widgets
        self.filter_text = widgets.Text(
            value="",
            description="Filter KO list",
            placeholder="Type to filter..."
        )
        self.ko_formatted_names_w = widgets.Select(
            options=self.ko_formatted_names,
            description="KO list",
            rows=40,
            layout=widgets.Layout(width='450px')
        )
        self.genomes_input = widgets.TagsInput(
            value=[],
            description="Genomes",
            allow_duplicates=False
        )
        self.ko_input = widgets.TagsInput(
            value=[],
            description="KEGG KOs",
            allow_duplicates=False
        )
        self.heatmap_output = widgets.Output()
        self.reset_button = widgets.Button(description="Reset Heatmap")
        
        # Attach behavior
        self.genomes_input.observe(self.update_heatmap, names="value")
        self.ko_input.observe(self.update_heatmap, names="value")
        self.filter_text.observe(self.filter_ko_list, names="value")
        self.ko_formatted_names_w.observe(self.add_to_ko_input, names="value")
        self.ko_formatted_names_w.value = None
        self.reset_button.on_click(self.reset_table_and_replot)
        self.updating_heatmap = False  # Add a flag to prevent duplicate updates

    def load_ko_list(self, ko_list_path, ko_synonyms_path):
        """Load and format the KO list with synonyms."""
        ko_list_df = pd.read_csv(ko_list_path, sep="\t", index_col=0)
        ko_list_df = ko_list_df.loc[:, ["definition"]]
        ko_list_df.columns = ["gene_name"]

        ko_synonyms = pd.read_csv(ko_synonyms_path, sep="\t", index_col=0, header=None, names=["synonyms"])
        ko_synonyms.fillna("", inplace=True)

        ko_list_df = ko_list_df.merge(ko_synonyms, left_index=True, right_index=True)
        ko_list_df["gene_name"] = ko_list_df["synonyms"] + ", " + ko_list_df["gene_name"]
        ko_list_df.drop(columns=["synonyms"], inplace=True)
        ko_list_df.index.name = "KO"
        return ko_list_df

    def read_df(self, sample):
        """Optimized function to read sample data."""
        return pd.read_csv(f"../../input_folder/dram_faa/{sample}/annotations.tsv", sep="\t")

    def count_sample_ko(self, annot_df, ko):
        """Optimized function to count KO occurrences."""
        return annot_df["ko_id"].eq(ko).sum()

    def update_matrix(self, genomes, genes):
        """Update the KO-genome occurrence matrix."""
        new_columns = [self.ko_list[gene] for gene in genes]
        self.df_count_ko = self.df_count_ko.reindex(columns=new_columns)

        for gene in genes:
            if self.ko_list[gene] not in self.df_count_ko.columns:
                self.df_count_ko[self.ko_list[gene]] = 0

        for genome in genomes:
            if genome not in self.df_count_ko.index:
                self.df_count_ko.loc[genome] = 0

        for genome in genomes:
            for gene in genes:
                gene_name = self.ko_list[gene]
                counts = self.count_sample_ko(self.read_df(genome), gene)
                self.df_count_ko.at[genome, gene_name] = counts

    def update_heatmap(self, change=None):
        """Update the heatmap visualization."""
        if self.updating_heatmap:
            return  # Skip if already updating
    
        self.updating_heatmap = True  # Set the flag
        with self.heatmap_output:
            clear_output(wait=True)
    
            # Process genomes_input to handle whitespace-separated values
            processed_genomes = []
            for genome in self.genomes_input.value:
                genome = re.sub(r"[\'\[\],]", " ", genome)
                if " " in genome:  # Check for whitespace
                    processed_genomes.extend(genome.split())  # Split and add the items in order
                else:
                    processed_genomes.append(genome)
    
            self.genomes_input.value = processed_genomes  # Update the value without altering order
            selected_genomes = self.genomes_input.value
    
            processed_kos = []
            for ko in self.ko_input.value:
                ko = re.sub(r"[\+\-,()]", " ", ko)
                if " " in ko:
                    kos = [k for k in ko.split() if re.match("^K[0-9]*", k)]
                    kos = remove_duplicates(kos)
                    processed_kos.extend(kos)
                else:
                    processed_kos.append(ko)
    
            self.ko_input.value = processed_kos
            selected_kos = self.ko_input.value
    
            self.update_matrix(selected_genomes, selected_kos)
    
            mk_max = self.df_count_ko.max().max()
            mk_val1 = 1 / mk_max if mk_max > 0 else 1
    
            num_rows, num_cols = self.df_count_ko.shape
            cell_size = 30  # Size per cell in pixels
            min_width = 400  # Minimum width for heatmap
            min_height = 300  # Minimum height for heatmap
    
            # Dynamically adjust the figure dimensions
            heatmap_width = max(num_cols * cell_size, min_width)
            heatmap_height = max(num_rows * cell_size, min_height)
            total_width = heatmap_width + 200  # Add 200 px for y-axis tick labels
            total_height = heatmap_height + 200  # Add 200 px for x-axis tick labels
    
            fig = px.imshow(self.df_count_ko, aspect="auto", text_auto=True)
            fig.update_layout(
                width=total_width,
                height=total_height,
                font=dict(size=18),
                margin=dict(l=200, t=50, r=50, b=200),  # Space for labels and a clean layout
            )
    
            tickvals = list(range(self.df_count_ko.shape[1]))
            xticktext = [re.sub(",.*", "", c) for c in self.df_count_ko.columns.tolist()]
            fig.update_xaxes(tickangle=90, ticktext=xticktext, tickvals=tickvals)
    
            fig.update_traces(xgap=1, ygap=1)
    
            colorscale = [
                (0.0, self.colors[0]),
                (mk_val1 - 0.25 * mk_val1, self.colors[0]),
                (mk_val1, self.colors[1]),
                (mk_val1 + 0.25 * mk_val1, self.colors[2]),
                (1, self.colors[2])
            ] if mk_val1 < 1 else [
                (0.0, self.colors[0]),
                (0.75, self.colors[1]),
                (1, self.colors[2])
            ]
            fig.update_coloraxes(colorscale=colorscale)
            fig.show(config={"editable": True})
    
        self.updating_heatmap = False  # Reset the flag

    def reset_table_and_replot(self, change=None):
        """Reset the matrix and refresh the heatmap."""
        self.df_count_ko = pd.DataFrame()
        selected_genomes = self.genomes_input.value
        selected_kos = self.ko_input.value
        new_columns = [self.ko_list[gene] for gene in selected_kos]
        self.df_count_ko = self.df_count_ko.reindex(columns=new_columns)
        self.update_matrix(selected_genomes, selected_kos)
        self.update_heatmap(None)

    def filter_ko_list(self, change):
        """Filter KO list based on user input."""
        filter_value = self.filter_text.value.lower()
        filtered_options = [
            ko for ko in self.ko_formatted_names if filter_value in ko.lower()
        ]
        self.ko_formatted_names_w.options = filtered_options

        if not filter_value:
            self.ko_formatted_names_w.value = None

    def add_to_ko_input(self, change):
        """Add selected KO to the input list."""
        selected_item = self.ko_formatted_names_w.value
        if selected_item:
            selected_ko = selected_item.split(":")[0]
            if selected_ko not in self.ko_input.value:
                self.ko_input.value = self.ko_input.value + [selected_ko]

    def display(self):
        """Display the dashboard."""
        layout = widgets.VBox([
            widgets.HBox([
                self.heatmap_output,
                widgets.VBox(),
                widgets.VBox([self.filter_text, self.ko_formatted_names_w, self.reset_button])
            ]),
            widgets.HTML(value="<b>Selected Genomes </b>"),
            self.genomes_input,
            widgets.HTML(value="<b>Selected KEGG KO</b>"),
            self.ko_input
        ])
        display(layout)


# Usage
dashboard = KOHeatmapDashboard()
dashboard.display()

In [14]:
import ipywidgets as widgets
from IPython.display import display
from Bio.KEGG import REST
import pandas as pd

class KEGGModuleDashboard:
    def __init__(self):
        # Step 1: Fetch KEGG modules and prepare data
        self.module_df = self.fetch_kegg_modules()
        self.modules_formatted_names = (
            self.module_df["module"] + ": " + self.module_df["name"]
        ).tolist()

        # Step 2: Define widgets
        self.filter_text = widgets.Text(
            value="",
            description="Filter modules",
            placeholder="Type to filter..."
        )

        self.modules_select_w = widgets.Select(
            options=self.modules_formatted_names,
            description="Modules list",
            rows=20,
            layout=widgets.Layout(width="450px")
        )

        self.module_info_output = widgets.Output()

        # Step 3: Attach widget behaviors
        self.filter_text.observe(self.filter_modules, names="value")
        self.modules_select_w.observe(self.display_module_info, names="value")

        # Step 4: Create the dashboard layout
        self.dashboard = widgets.HBox(
            [widgets.VBox([self.filter_text, self.modules_select_w]), self.module_info_output]
        )

    def fetch_kegg_modules(self):
        """Fetch the list of KEGG modules and return as a DataFrame."""
        module_list_data = REST.kegg_list("module").read()
        return pd.DataFrame(
            [line.split("\t") for line in module_list_data.strip().split("\n")],
            columns=["module", "name"]
        )

    def filter_modules(self, change):
        """Update the options in the module select widget based on filter text."""
        search_text = change["new"]
        if search_text == "":
            filtered_modules = self.modules_formatted_names
        else:
            filtered_modules = [
                name for name in self.modules_formatted_names
                if search_text.lower() in name.lower()
            ]
        self.modules_select_w.options = filtered_modules

    def display_module_info(self, change):
        """Fetch and display module information for the selected module."""
        self.module_info_output.clear_output()
        selected_module = change["new"].split(":")[0]  # Extract module ID
        if selected_module:
            with self.module_info_output:
                module_info = REST.kegg_get(f"module:{selected_module}").readlines()
                for l in module_info:
                    if "NAME" in l:
                        name = l
                        break
                name = re.sub("NAME *", "", name)
                print(f"{selected_module}: {name}")
                module_info_description = "".join(module_info)
                print(module_info_description)

    def display(self):
        """Display the dashboard."""
        display(self.dashboard)

# Create and display the KEGG Module Dashboard
kegg_dashboard = KEGGModuleDashboard()
kegg_dashboard.display()


# KIV below